In [ ]:
from osgeo import gdal
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
try:
    from dbfread import DBF
except ImportError:
    !pip install dbfread
# from scipy.stats import pearsonr
import scipy.stats as stats
import seaborn as sns
import itertools
import statsmodels.formula.api as smf

# Import Data

In [ ]:
X_his_root = 'Processed Data/IndependentVariableX/HistoricalData'
def readData(path):
     # file root
    table = DBF(path, encoding='GBK')
    df = pd.DataFrame(iter(table))
    return df
POP_his_df={}
LULC_his_df={}
LST_his_df={}
DEM_his_df={}

In [ ]:
for i in range(2005,2021,5):
    path1=X_his_root+'/Pop_HIS_Table/Proj_usa_ppp_'+str(i)+'_UNadj_Table.dbf'
    path2='Processed Data/因变量LST/LST'+str(i)+'.dbf'# LST
    if i==2010:
        path3=X_his_root+'/LULC_HIS_Table/reclass_nlcd_'+str(i)+'.dbf'
    else:
        path3=X_his_root+'/LULC_HIS_Table/reclass_nlcd_'+str(i+1)+'.dbf'
    POP_his_df[str(i)]=readData(path1)
    LST_his_df[str(i)]=readData(path2)
    LULC_his_df[str(i)]=readData(path3)
    DEM_his_df[str(i)]=readData('Processed Data/IndependentVariableX/HistoricalData/DEM_Table/DEM_Table.dbf')

In [ ]:
# DEM_his_df=readData('Processed Data/IndependentVariableX/HistoricalData/DEM_Table/DEM_Table.dbf')

# Clean Data

In [ ]:
len(DEM_his_df['2005'])

34258

In [ ]:
POP_his_df['2005'].head(3)

,FID_,COUNT,AREA,SUM
0,0,100.0,1000000.0,44.800169
1,1,100.0,1000000.0,2.065253
2,2,100.0,1000000.0,88.943340


In [ ]:
POP_his_df['2010'].describe()

,FID_,COUNT,AREA,SUM
count,32527.000000,32527.000000,32527.000000,32527.000000
mean,17929.078058,98.465552,984655.516955,164.845943
std,9498.150608,8.615041,86150.413335,496.928144
min,0.000000,1.000000,10000.000000,0.000000
25%,9861.500000,100.000000,1000000.000000,4.263271
50%,17993.000000,100.000000,1000000.000000,22.164169
75%,26124.500000,100.000000,1000000.000000,92.479616
max,34257.000000,100.000000,1000000.000000,11186.639513


In [ ]:
len(POP_his_df['2005'])

32527

列名SUM表示此格网内人口总数

In [ ]:
LULC_his_df['2005'].head(3)

,FID_,VALUE_1,VALUE_2,VALUE_3,VALUE_4,VALUE_5,VALUE_6,VALUE_7,VALUE_8
0,0,297900.0,3600.0,108000.0,63900.0,0.0,90000.0,0.0,416700.0
1,1,25200.0,0.0,51300.0,0.0,0.0,216900.0,0.0,686700.0
2,2,434700.0,0.0,140400.0,4500.0,0.0,13500.0,0.0,416700.0


In [ ]:
LULC_his_df['2005'].describe()

,FID_,VALUE_1,VALUE_2,VALUE_3,VALUE_4,VALUE_5,VALUE_6,VALUE_7,VALUE_8
count,34258.000000,3.425800e+04,3.425800e+04,34258.000000,3.425800e+04,34258.000000,3.425800e+04,34258.000000,3.425800e+04
mean,17128.500000,1.823214e+05,7.757843e+04,156059.679491,4.644216e+04,3180.343277,4.436418e+05,3944.599802,8.682218e+04
std,9889.577097,2.411756e+05,2.357550e+05,187904.780206,1.243183e+05,22771.232793,3.471851e+05,16229.791803,1.704936e+05
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,8564.250000,0.000000e+00,0.000000e+00,28800.000000,0.000000e+00,0.000000,9.450000e+04,0.000000,0.000000e+00
50%,17128.500000,5.310000e+04,0.000000e+00,84600.000000,1.800000e+03,0.000000,4.149000e+05,0.000000,1.350000e+04
75%,25692.750000,3.078000e+05,9.900000e+03,203400.000000,2.070000e+04,0.000000,7.749000e+05,1800.000000,8.730000e+04
max,34257.000000,1.033200e+06,1.040400e+06,982800.000000,1.035000e+06,543600.000000,1.040400e+06,522000.000000,1.040400e+06


In [ ]:
len(LULC_his_df['2010'])

32565

列名Value 1等表示分类（参考分类对照表），列FID作为唯一标识符。表格内的值表示一个格网内此Landuse类型的占地面积（平方米）。

The column name Value 1, etc. indicates the classification (refer to the Classification Comparison Table) and the column FID is used as a unique identifier. The values in the table represent the area (in square meters) of this Landuse type within a grid.

In [ ]:
DEM_his_df['2005'].head(3)

,FID_,COUNT,AREA,MEAN
0,0,1122.0,1009800.0,14.010071
1,1,1089.0,980100.0,13.740351
2,2,1089.0,980100.0,13.823337


In [ ]:
DEM_his_df['2005'].describe()

,FID_,COUNT,AREA,MEAN
count,34258.000000,34258.000000,3.425800e+04,34258.000000
mean,17128.500000,1111.116323,1.000005e+06,223.690762
std,9889.577097,22.226164,2.000355e+04,209.153268
min,0.000000,1089.000000,9.801000e+05,-1.298297
25%,8564.250000,1089.000000,9.801000e+05,29.589671
50%,17128.500000,1122.000000,1.009800e+06,157.038539
75%,25692.750000,1122.000000,1.009800e+06,394.806578
max,34257.000000,1156.000000,1.040400e+06,1130.898398


In [ ]:
LST_his_df['2005'].head(3)

,FID_,COUNT,AREA,MEAN
0,0,1.0,1000000.0,302.282
1,1,1.0,1000000.0,301.188
2,2,1.0,1000000.0,301.730


In [ ]:
LST_his_df['2020'].describe()

,FID_,COUNT,AREA,MEAN
count,34258.000000,34258.0,34258.0,32265.000000
mean,17128.500000,1.0,1000000.0,301.741838
std,9889.577097,0.0,0.0,3.708273
min,0.000000,1.0,1000000.0,292.692000
25%,8564.250000,1.0,1000000.0,298.662000
50%,17128.500000,1.0,1000000.0,301.678000
75%,25692.750000,1.0,1000000.0,303.968000
max,34257.000000,1.0,1000000.0,314.885000


POP中,LULC中数据不全，LST里有缺失值

## 删除缺失值


In [ ]:
index_l=list(set(POP_his_df['2005']['FID_']) & set(LULC_his_df['2005']['FID_']))

In [ ]:
LULC_his_df[str(i)].iloc[:,1:].sum(axis=1)

0         980100.0
1         980100.0
2        1009800.0
3         980100.0
4         980100.0
           ...    
34253    1009800.0
34254    1009800.0
34255     980100.0
34256     980100.0
34257    1009800.0
Length: 34258, dtype: float64

In [ ]:
LULC_his_df['2005'][(LULC_his_df['2005']['VALUE_3']+LULC_his_df['2005']['VALUE_4'])/LULC_his_df['2005']['Row_sum']>=0.4]

KeyError: 'Row_sum'

In [ ]:
for i in range(2005,2021,5):
    LST_his_df[str(i)].dropna(inplace=True)
    LST_his_df[str(i)].reset_index(drop=True,inplace=True)

    LULC_his_df[str(i)]['Row_sum'] = LULC_his_df[str(i)].iloc[:,1:].sum(axis=1)  # 对1~8列按行求和，生成新列
    LULC_his_df[str(i)]=LULC_his_df[str(i)][(LULC_his_df[str(i)]['Row_sum']>=980100)]
    LULC_his_df[str(i)]=LULC_his_df[str(i)][(LULC_his_df[str(i)]['VALUE_3']+LULC_his_df[str(i)]['VALUE_4'])/LULC_his_df[str(i)]['Row_sum']>=0.4]
    LULC_his_df[str(i)].reset_index(drop=True,inplace=True)

    index_l=list(set(POP_his_df[str(i)]['FID_']) & set(LULC_his_df[str(i)]['FID_'])& set(LST_his_df[str(i)]['FID_'])& set(DEM_his_df[str(i)]['FID_']))
    LST_his_df[str(i)]=LST_his_df[str(i)][LST_his_df[str(i)]['FID_'].isin(index_l)].reset_index(drop=True)
    LULC_his_df[str(i)]=(LULC_his_df[str(i)][LULC_his_df[str(i)]['FID_'].isin(index_l)]).reset_index(drop=True)
    POP_his_df[str(i)]=(POP_his_df[str(i)][POP_his_df[str(i)]['FID_'].isin(index_l)]).reset_index(drop=True)
    DEM_his_df[str(i)] = (DEM_his_df[str(i)][DEM_his_df[str(i)]['FID_'].isin(index_l)]).reset_index(drop=True)

## Describe Data

In [ ]:
for i in ['2005','2010','2015','2020']:
    print(i+"\n")
    print("LST_his_df\n")
    print(LST_his_df[i].describe())
    print("LULC_his_df\n")
    print(LULC_his_df[i].describe())
    print("POP_his_df\n")
    print(POP_his_df[i].describe())
    print("DEM_his_df\n")
    print(DEM_his_df[i].describe())
    print("\n\n")

## LST plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x_axis_data = [2005,2010,2015,2020] #x
y_axis_data = [68,69,79,71,80,70,66] #y

plt.plot(x_axis_data, y_axis_data, 'b*--', alpha=0.5, linewidth=1, label='acc')#'bo-'表示蓝色实线，数据点实心原点标注
## plot中参数的含义分别是横轴值，纵轴值，线的形状（'s'方块,'o'实心圆点，'*'五角星   ...，颜色，透明度,线的宽度和标签 ，

plt.legend()  #显示上面的label
plt.xlabel('time') #x_label
plt.ylabel('number')#y_label

#plt.ylim(-1,1)#仅设置y轴坐标范围
plt.show()


# Correlation analysis

Correlation analysis: Calculate Pearson correlation of LST vs pop, DEM and LandUse Type Percentage for each pixel. And compare the difference between urban area and rural. And choose the most correlated variables as features in the model. (((如果不是对每个pixel做回归，而是对整体的城市区域和农村区域， if 残差具有空间异质性可以验证一下，LST可能会有空间自相关（Moran指数. 有的话用地理探测器选择因素。

## 2020

In [ ]:
# for i in range(2005,2021,5):
# POP_his_df={}
# LULC_his_df={}
# LST_his_df={}
dataByYear={}

for i in ['2005','2010','2015','2020']:
    dataByYear[i]=pd.DataFrame({
        "LST":LST_his_df[i]['MEAN'],
        "LULC_1":LULC_his_df[i]["VALUE_1"]/LULC_his_df[i]["Row_sum"],
        "LULC_2":LULC_his_df[i]["VALUE_2"]/LULC_his_df[i]["Row_sum"],
        "LULC_3":LULC_his_df[i]["VALUE_3"]/LULC_his_df[i]["Row_sum"],
        "LULC_4":LULC_his_df[i]["VALUE_4"]/LULC_his_df[i]["Row_sum"],
        "LULC_5":LULC_his_df[i]["VALUE_5"]/LULC_his_df[i]["Row_sum"],
        "LULC_6":LULC_his_df[i]["VALUE_6"]/LULC_his_df[i]["Row_sum"],
        "LULC_7":LULC_his_df[i]["VALUE_7"]/LULC_his_df[i]["Row_sum"],
        "LULC_8":LULC_his_df[i]["VALUE_8"]/LULC_his_df[i]["Row_sum"],
        "DEM":DEM_his_df[i]['MEAN'],
        "POP":POP_his_df[i]['SUM'],
        "year":np.full(len(DEM_his_df[i]), int(i))
    })
fullData=pd.concat([dataByYear['2005'],dataByYear['2010'],dataByYear['2015'],dataByYear['2020']])
# data_2020=pd.DataFrame({
#     "LST":LST_his_df['2020']['MEAN'],
#     "LULC_1":LULC_his_df['2020']["VALUE_1"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_2":LULC_his_df['2020']["VALUE_2"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_3":LULC_his_df['2020']["VALUE_3"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_4":LULC_his_df['2020']["VALUE_4"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_5":LULC_his_df['2020']["VALUE_5"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_6":LULC_his_df['2020']["VALUE_6"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_7":LULC_his_df['2020']["VALUE_7"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_8":LULC_his_df['2020']["VALUE_8"]/LULC_his_df['2020']["Row_sum"],
#     "DEM":DEM_his_df['2020']['MEAN'],
#     "POP":POP_his_df['2020']['SUM'],
#     "year":np.full(len(DEM_his_df['2020']), 2020)
# })
# data_2020=pd.DataFrame({
#     "LST":LST_his_df['2020']['MEAN'],
#     "LULC_1":LULC_his_df['2020']["VALUE_1"],
#     "LULC_2":LULC_his_df['2020']["VALUE_2"],
#     "LULC_3":LULC_his_df['2020']["VALUE_3"],
#     "LULC_4":LULC_his_df['2020']["VALUE_4"],
#     "LULC_5":LULC_his_df['2020']["VALUE_5"],
#     "LULC_6":LULC_his_df['2020']["VALUE_6"],
#     "LULC_7":LULC_his_df['2020']["VALUE_7"],
#     "LULC_8":LULC_his_df['2020']["VALUE_8"],
#     "DEM":DEM_his_df['2020']['MEAN'],
#     "POP":POP_his_df['2020']['SUM'],

# })
List=list(itertools.combinations(data_2020.keys(),2))


In [ ]:
fullData.describe()

In [ ]:
Corr=fullData.corr(method='pearson')
Corr

In [ ]:
# sns.heatmap(Corr_2020,linewidths=0.8,vmax=1.0, square=True,linecolor='white', annot=True)

In [ ]:
p_fullData= pd.DataFrame(columns=Corr.columns,index=Corr.index)
p_fullData

In [ ]:
for i in List:
    i=i+stats.pearsonr(fullData[i[0]],fullData[i[1]])
    p_fullData[i[0]][i[1]]=i[3]
    print(i)

In [ ]:
pd.options.display.precision = 4
p_fullData

In [ ]:
f1='LST ~ '+'+'.join(fullData.columns[1:])
f1

VIF值越大则共线性问题越明显，通常以10作为判断边界。当VIF<10不存在多重共线性；当10<=VIF<100,存在较强的多重共线性；当VIF>=100, 存在严重多重共线性。

In [ ]:
reg = smf.ols(formula=f1, data = fullData)
results = reg.fit()
from statsmodels.stats.outliers_influence import variance_inflation_factor

exog = reg.exog #获取自变量矩阵（第一列为截距项）
exog_names = reg.exog_names #获取自变量名称（第一个为截距项）

vif = []
for i in range(exog.shape[1]-1):
    vif.append(variance_inflation_factor(exog,i+1))
    print(f'{exog_names[i+1]} 的方差膨胀因子VIF = {vif[i]}')

LULC_1, LULC_6 >100. so delete L6

In [ ]:
f2='LST ~ '+'+'.join(fullData.columns[1:6].append(fullData.columns[7:]))
f2

In [ ]:
reg2 = smf.ols(formula=f2, data = fullData)
results = reg2.fit()
from statsmodels.stats.outliers_influence import variance_inflation_factor

exog = reg2.exog #获取自变量矩阵（第一列为截距项）
exog_names = reg2.exog_names #获取自变量名称（第一个为截距项）

vif = []
for i in range(exog.shape[1]-1):
    vif.append(variance_inflation_factor(exog,i+1))
    print(f'{exog_names[i+1]} 的方差膨胀因子VIF = {vif[i]}')


## Reprojection

## Resample